# Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We do not recommand to run the notebook from within the Leaspy repo (to prevent it from interfering with Leaspy git versioning).
Therefore, copy paste the start folder elsewhere, and replace the `project_path` with the path to the leaspy folder.

In [ ]:
# these 2 lines are not needed if you installed leaspy with pip
leaspy_path = os.path.join(os.getcwd(), '..', '..')
sys.path.append(leaspy_path)

In [ ]:
from leaspy import Leaspy, Data, AlgorithmSettings, IndividualParameters, __watermark__
# Watermark trace with all packages versions
__watermark__

# Step 1. Data

You have to store your data into a `Data` object.

### Option #1
You can load your data directly from a csv file, whose rows corresponds to subject visits, each column being a feature - except for the `ID` and `TIME` column. This result in multiple rows per subject. The input format has therefore to follow the following rules:
- A column named `ID`: corresponds to the subject indices
- A columns named `TIME`: corresponds to the subject's age at the corresponding visit
- One column per feature
- Each row is a visit, therefore the concatenation of the subject ID, the patient age at which the corresponding visit occured, and then the feature values.

Here is an example :

| ID | TIME | Feature 1 | Feature 2 | Feature 3 | Feature 4
| --- | --- | --- | --- | --- | --- |
| 309 | 70.4 | 25 | 0.12 | -0.19 | 3 |
| 309 | 73.4 | 10 | 0.13 | NaN | 1.3 |
| 309 | 75.7 | 24.3 | 0.0 | -0.44 | 0.8 |
| 40 | 60.1 | 5 | NaN | -0.12 | 2.3 |
| 40 | 62.5 | 23.4 | 0.8 | -0.142 | 0.94 |
| 918 | 88 | 9.3 | 0.9 | -0.3 | 0.23 |
| 918 | 92.3 | 33 | 0.88 | -0.423 | NaN |
| 918 | 95.3 | 34 | 0.58 | -0.523 | 0.34 |

You can find additional example in the `inputs` folder: `data.csv` and `data_univariate.csv`

You probably noticed that there are NaN: do not worry, Leaspy can handle them ;)

In [ ]:
data = Data.from_csv_file(os.path.join('inputs', 'data_normalized.csv'))

### Option #2

However, there are chances that your csv files contains data that either you don't want or that don't fit the model you are going to use. For instance, to build a logistic curve model, you need data that are normalized between 0 and 1 and that are increasing with time. Moreover, to calibrate the progression model, **we highly recommend to keep subjects that have been seen at least two times**. 

For that reason, you can load your data from a dataframe that you have preprocessed with pandas methods.

Let's seen an example where:
- Feature 1 is increasing through time and is between 0 and 1: nothing to do
- Feature 2 is increasing through time and is between 0 and 100: need to normalize between 0 and 1
- Feature 3 is decreasing through time and is between 0 and 1: need to revert the temporal progression to be increasing
- Feature 4 is decreasing through time and is between 60 and 0: need to revert and to normalize between 0 and 1

In [ ]:
df = pd.read_csv(os.path.join('inputs', 'data.csv'))

# —— Feature 1
df['Feature 1'] = df['Feature 1']

# —— Feature 2
df['Feature 2'] = df['Feature 2']/100

# —— Feature 3
df['Feature 3'] = -df['Feature 3']+1

# —— Feature 4
df['Feature 4'] = -df['Feature 4']/60+1

# —— Let's select patient with at least two visits
df = df.set_index(['ID', 'TIME'])
indices = [idx for idx in df.index.unique('ID') if df.loc[idx].shape[0] >= 2]
df = df[df.index.get_level_values(0).isin(indices)]

# —— Store the data into a Data object
data = Data.from_dataframe(df)

**Remark & Tip:** while some data have natural minimum and maximum values (inducing ceiling and floor effects), other don't have natural bounds. In that case, normalize as best as you can so that you don't have values (current or future) higher than 1 or lower than 0. You can check that with `Feature 4` that does not reach maximum values or `Feature 1` that starts above minimum value

In [ ]:
fts_min_max = df.agg(['min','max'])

assert (fts_min_max.loc['min'] >= 0).all()
assert (fts_min_max.loc['max'] <= 1).all()

pd.options.display.float_format = '{:.2f}'.format
fts_min_max

# Step 2. Settings

To run any algorithm, you need to specify the settings of the related algorithm thanks to the `AlgorithmSettings` object. 

### Option #1

To ease Leaspy's usage for new users, we specified default values for each algorithm. Therefore, the name of the algorithm used is enough to run it. The one you need to fit your progression model is `mcmc_saem`. Let's see an example:

In [ ]:
algo_settings = AlgorithmSettings('mcmc_saem', 
                                  n_iter=3000,           # n_iter defines the number of iterations
                                  progress_bar=True)     # To display a nice progression bar during calibration

If you want to store the different logs of the model during the iterations, you can use the following method with the path of the folder where the logs will be stored

In [ ]:
algo_settings.set_logs(
    path='outputs/logs', # Creates a logs file ; if existing, ask if rewrite it
    save_periodicity=50, # Saves the values in csv files every N iterations
    console_print_periodicity=100, # Displays logs in the console/terminal every N iterations, or None
    plot_periodicity=1000, # Generates the convergence plots every N iterations
    overwrite_logs_folder=True # if True and the logs folder already exists, it entirely overwrites it
)

# Step 3. Fit model

Now that you have your data and your settings, you can run the model. You first need to choose the type of progression shape you want to give to your data. You can either choose logistic or linear (soon exponential) with the possibility to enforce a "parallelism" between the features. The dedicated names are  :

`logistic`, `logistic_parallel`, `linear` and `linear_parallel`. You can also call the `univariate` model which is a single logistic function that you can run on the `input/data_univariate.csv`.

Note : the model might rely on hyperparameters that you can define as shown below. There are optional. The most important one is the `source_dimension`. To be short, if you consider `N` features, these can be reordered as not everyone follows the same relative ordering of events. However, all reordering are not likely to exist, and there are some common patterns within a population. Therefore, the `source_dimension` is in a way a subspace that defines the degrees of freedom that you have to reorder you sequence of events. Still not clear? That's fine. It's related to the `space-shifts` in papers described in the repo. But don't worry if you didn't get it yet. Make it equal to somewhat higher than the log/square-root of the number of features.

In [ ]:
leaspy = Leaspy("logistic", 
                source_dimension=2, # Optional
                noise_model='gaussian_diagonal', # Optional: To get a noise estimate per feature keep it this way (default)
                )

Congrats! You are now ready to run the model

In [ ]:
leaspy.fit(data, settings=algo_settings)

### Did it run well?

Before assuming that the model is estimated, you have to check that the convergence went went. For that, you can look  the at the convergence during the iterations. To do so, you can explore the `logs` folder that shows the model convergence during the iterations. The first thing to look at is probably the `plots/convergence_1.pdf` and `plots/convergence_2.pdf` files : a run has had enough iterations to converge if the last 20 or even 30% of the iterations were stable for all the parameters. If not, you should provably rerun it with more iterations. 

In [ ]:
from IPython.display import IFrame
IFrame('./outputs/logs/plots/convergence_1.pdf', width=990, height=670)

Then, you can check out the parameters of the model that are stored here : 

In [ ]:
leaspy.model.parameters

They are probably not straightfoward for now. The most important one is probably `noise_std`. It corresponds to the standard deviation of the error for a given feature. The smallest, the better - up to the lower bound which is the intrinsic noise in the data. Let's display it:

In [ ]:
noise = leaspy.model.parameters['noise_std']
features = leaspy.model.features

print('Standard deviation of the residual noise for feature:')
for n, f in zip(noise, features):
    print(f'- {f}: {n:.2%}')

**Remark:** Usually, cognitive measurements have an intrinsic error (computed on test-retest exams) between 5 and 10%.

### Save and load the model

There are many reasons why one might want to save the output parameters of the model. To do so, just do: 

In [ ]:
leaspy.save("outputs/model_parameters.json")

Later, you can load the model if needed.

In [ ]:
leaspy = Leaspy.load('outputs/model_parameters.json')

### Display the average model

Now that we have sufficient evidence that the model has converged, let's output what the average progression look like! 

First, let's detail a bit what we are going to represent. We are going to display a trajectory: it corresponds to the temporal progression of the biomarkers. There is not only one trajectory for a cohort, as each subject has his or her own specific trajectory, meaning his or her disease progression. Each of these individual trajectories are rely on individual parameters that are subject-specific. We will see what this individual parameters a bit later, do not worry. For now, let's stick to the "average" trajectory.

So what does the average trajectory corresponds to? The average trajectory corresponds to a "virtual" patient whose individual parameters are the average individual parameters. And these averages are already estimated during the calibration.

So let's get these average individual parameters and store them in the `IndividualParameters` object that stores a collection of individual parameter, for many subjects. Here, it is used for only one subject, called `average`.

In [ ]:
# —— Get the average individual parameters
mean_xi = leaspy.model.parameters['xi_mean'].numpy()
mean_tau = leaspy.model.parameters['tau_mean'].numpy()
mean_source = leaspy.model.parameters['sources_mean'].numpy().tolist()
number_of_sources = leaspy.model.source_dimension
mean_sources = [mean_source]*number_of_sources

# —— Store the average individual parameters in a dedicated object
average_parameters = {
    'xi': mean_xi,
    'tau': mean_tau,
    'sources': mean_sources
}

ip_average = IndividualParameters()
ip_average.add_individual_parameters('average', average_parameters)

The second very important function - after `leaspy.fit()` - is `leaspy.estimate`. Given some individual parameters and timepoints, the function estimates the values of the biomarkers at the given timepoints which derive from the individual trajectory encoded thanks to the individual parameters.

You can check out the documentation and run it : 

In [ ]:
?leaspy.estimate

In [ ]:
timepoints = np.linspace(55, 105, 100)
values = leaspy.estimate({'average': timepoints}, ip_average)

def plot_trajectory(timepoints, reconstruction, observations=None, *, 
                    xlabel='Reparametrized age', ylabel='Normalized feature value'):

    if observations is not None:
        ages = observations.index.values
    
    plt.figure(figsize=(10, 5))
    plt.ylim(0, 1)
    colors = ['#003f5c', '#7a5195', '#ef5675', '#ffa600']
    
    for c, name, val in zip(colors, leaspy.model.features, reconstruction.T):
        plt.plot(timepoints, val, label=name, c=c, linewidth=3)
        if observations is not None:
            plt.plot(ages, observations[name], c=c, marker='o', markersize=12, 
                     linewidth=1, linestyle=':')
    
    plt.xlim(min(timepoints), max(timepoints))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.legend()
    plt.show()
    
plot_trajectory(timepoints, values['average'])

# Step 4. Personalize the model to individual data

The personalization procedure allows to estimate the individual parameters that allows to modify the average progression to individual observations. The variations from the average trajectory to the individual one are encoded within three individual parameters : 
- $\alpha_i = \exp(\xi_i)$ : the acceleration factor, that modulates the speed of progression : $\alpha_i > 1$ means faster, $\alpha_i < 1$ means slower than the average progression
- $\tau_i$ : the time shift which delays the progression in a given number of years. It has to be compared to  `tau_mean` $ = \bar{\tau} $  which is in the model parameters above. In fact, $ \tau_i \sim N( \bar{\tau}, \sigma_{\tau}^2)$ , so $\tau_i > \bar{\tau}$ means that the patient has a disease that starts later than average, while $\tau_i < \bar{\tau}$ means that the patient has a disease that starts earlier than average
- $w_i = (w_1, ..., w_N)$ ($N$ being the size of the feature space) : the space-shift  which might, for a given individual, change the ordering of the conversion of the different features, compared to the mean trajectory.

In a nutshell, the $k$-th feature at the $j$-th visit of subject $i$, which occurs at time $t_{ij}$ writes: 

$$y_{ijk} = f_\theta ( w_{ik}+ \exp(\xi_i) * (t_{ij} - \tau_i) ) + \epsilon_{ijk}$$

This writing is not exactly correct but helps understand the role of each individual parameters.

**[ Advanced ]** Remember the `sources`, or the `source_dimension`? Well, $w_i$ is not estimated directly, but rather thanks to a Independant Component Analysis, such that $w_i = A s_i$ where $s_i$ is of size $N_s$ = `source_dimension`. See associated papers for further details.

Now, let's estimate these individual parameters. The procedure relies on the `scipy_minimize` algorithm that you have to define (or to load from an appropriate json file) : 

In [ ]:
# Option 1
settings_personalization = AlgorithmSettings('scipy_minimize', use_jacobian=True)

# Option 2
settings_personalization = AlgorithmSettings.load('inputs/algorithm_settings_personalization.json')

Here is the third most important function of leaspy : `leaspy.personalize`. It estimates the individual parameters for the data you provide:

In [ ]:
?leaspy.personalize

In [ ]:
ip = leaspy.personalize(data, settings_personalization)

Note here that you can personalize your model on patients that have only one visit! You don't have to use the same `data` as previously. Especially, you can here personalize your model with patients that have only one visit.

Once the personalization is done, you can check the different functions that the `IndividualParameters` provides (you can load and save them, transform them to dataframes, etc) : 

In [ ]:
?ip

Now, let's see what you can do with the individual parameters.

# Step 5. Impute missing values & predict individual trajectories

The individual parameters entirely defines the individual trajectory, and thus, the biomarker values at any time. So you can reconstruct the individual biomarkers at different ages. 

You can reconstruct your observations at seen ages, i.e. at visits that have been used to personalize the model. There are two reasons you might want to do that:
- see how well the model fitted individual data
- impute missing values: as Leaspy handles missing values, it can then reconstruct them (note that this reconstruction will be noiseless)

To do so, let's first retrieve the observations of subject '1634-S2-1' in the initial dataset. You can also get his/her individual parameters as shown here:

In [ ]:
subject_id = '1634-S2-1'
observations = df.loc[subject_id]
print(f'Seen ages: {observations.index.values}')
print(ip[subject_id])

Now, as done with the average trajectory, let's estimate the trajectory for this patient.

**Remark**: The `estimate` first argument is a dictionary, so that you can estimate the trajectory of multiple individuals simultaneously (as long as the individual parameters of all your queried patients are in `ip`.

In [ ]:
timepoints = np.linspace(77, 85, 100)
reconstruction = leaspy.estimate({subject_id: timepoints}, ip)

plot_trajectory(timepoints, reconstruction[subject_id], observations, xlabel='Age')

From this plot, we clearly see that the estimations at late ages, above 82 years old are clearly prediction. To estimate the quality of the prediction, you can hide some future visits in the calibration and personalization procedure, then estimate the values at these visits and see how good it performs by comparing the predicting value compared to the hidden one.

# Step 6. Further Analysis

Besides prediction, the individual parameters are interesting in the sense that they provide meaningful and interesting insights about the disease progression. For that reasons, these individual parameters can be correlated to other cofactors. Let's seen an example.

Let's consider that you have a covariate `Cofactor 1` that encodes a genetic status: 1 if a specific mutation is present, 0 otherwise. Now, let's see if this mutation has an effect on the disease progression: 

In [ ]:
# —— Convert individual parameters to dataframe
df_ip = ip.to_dataframe()

# —— Merge with cofactors
cofactor = pd.read_csv('inputs/cofactor.csv', index_col=['ID'])
df_ip = df_ip.join(cofactor)

# —— Separate the individual parameters with respect to the cofactor
carriers = df_ip[df_ip['Cofactor 1'] == 0.]
non_carriers = df_ip[df_ip['Cofactor 1'] == 1.]

def plot_histo(title, ft, bins_nb=10):
    
    # compute bins (same for 2 carriers & non-carriers)
    _, bins = np.histogram(df_ip[ft], bins=bins_nb)
    
    plt.title(title)
    plt.hist(carriers[ft], density=True, bins=bins, edgecolor='white', label='Carriers')
    plt.hist(non_carriers[ft], density=True, bins=bins, alpha=0.6, edgecolor='white', label='Non carriers')
    plt.legend()
    plt.show()

# —— Plot the time shifts in carriers and non-carriers
plot_histo('Time shift histogram', 'tau')

# —— Plot the acceleration factor in carriers and non-carriers
plot_histo('Log-Acceleration factor histogram', 'xi')


It seems that the mutation has an effect on the disease onset, but not on its pace. Let's verify it with a statistical test

In [ ]:
from scipy import stats

# —— Student t-test (under the asumption of a gaussian distribution only)
print(stats.ttest_ind(carriers['tau'], non_carriers['tau']))
print(stats.ttest_ind(carriers['xi'], non_carriers['xi']))

# —— Mann-withney t-test
print(stats.mannwhitneyu(carriers['tau'], non_carriers['tau']))
print(stats.mannwhitneyu(carriers['xi'], non_carriers['xi']))

It is now clear that the mutation has an effect on the disease onset, but not on its pace.



# The end of this Notebook. But the beginning of your explorations.

You've now seen most of the applications that you can run with Leaspy. There are many more to discover based on your experimental settings and analysis. But it's time for you to try it on your own and to enjoy the package.

Have fun! 